In [1]:
import os
import pandas as pd
import re

In [2]:
# Read and Load files from miscellaneous

folder_path = "./miscellaneous"
files = [f for f in os.listdir(folder_path) if f.startswith("results_raw")]

dataframes = [] # archive processed data by file

for file in files:
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        # Store content for later processing
        dataframes.append({"filename": file, "content": lines})

In [3]:
# Spread Header from Body
for df in dataframes:
    content = df["content"]
    header_start = content.index("HEADER\n")
    body_start = content.index("BODY\n")
    
    header_data = content[header_start + 1 : body_start] # slicing between HEADER and BODY
    body_data = content[body_start + 1 : ] # From BODY until the end of the file
    
    # Remove blank lines and extra spaces
    header_data = [line.strip() for line in header_data if line.strip()]
    body_data = [line.strip() for line in body_data if line.strip()]
    
    df["header"] = header_data
    df["body"] = body_data
    

### Process HEADER
1. Extract the relevant fields like `numero_sorteo`, `primer_premio`, `fecha_sorteo`, etc., using regular expressions.
2. Create a DataFrame for the "sorteos" table.

In [4]:
sorteos_data = []
for df in dataframes:
    header = df["header"]
    numero_sorteo = re.search(r"NO. (\d+)", header[0]).group(1)
    tipo_sorteo = re.search(r"SORTEO (\w+)", header[0], re.IGNORECASE).group(1)
    fecha_sorteo = re.search(r"FECHA DEL SORTEO: ([\d/]+)", " ".join(header)).group(1)
    fecha_caducidad = re.search(r"FECHA DE CADUCIDAD: ([\d/]+)", " ".join(header)).group(1)
    premios = re.search(r"PRIMER PREMIO (\d+) \|\|\| SEGUNDO PREMIO (\d+) \|\|\| TERCER PREMIO (\d+)", " ".join(header))
    primer_premio, segundo_premio, tercer_premio = premios.groups()
    reintegros = re.search(r"REINTEGROS ([\d, ]+)", " ".join(header)).group(1).replace(" ", "")
    
    sorteos_data.append({
        "numero_sorteo": numero_sorteo,
        "tipo_sorteo": tipo_sorteo,
        "fecha_sorteo": fecha_sorteo,
        "fecha_caducidad": fecha_caducidad,
        "primer_premio": primer_premio,
        "segundo_premio": segundo_premio,
        "tercer_premio": tercer_premio,
        "reintegros": reintegros
    })

sorteos_df = pd.DataFrame(sorteos_data)
sorteos_df

,numero_sorteo,tipo_sorteo,fecha_sorteo,fecha_caducidad,primer_premio,segundo_premio,tercer_premio,reintegros
0,390,EXTRAORDINARIO,08/06/2024,09/12/2024,16780,46129,77885,"0,9,5"
1,3047,ORDINARIO,15/06/2024,16/12/2024,45100,74061,53248,"0,1,8"
2,3048,ORDINARIO,22/06/2024,23/12/2024,74569,47785,20555,"9,5,5"
3,3049,ORDINARIO,29/06/2024,30/12/2024,08241,29870,78927,"1,0,7"
4,3050,ORDINARIO,06/07/2024,06/01/2025,72487,10469,36580,"7,9,0"
5,3051,ORDINARIO,13/07/2024,13/01/2025,00967,32102,41229,"7,2,9"
6,391,EXTRAORDINARIO,21/07/2024,21/01/2025,01095,96032,12592,"5,2,2"
7,3052,ORDINARIO,27/07/2024,27/01/2025,27844,54823,06733,"4,3,3"


### Process BODY
1. Split the data into `premios`, `terminaciones`, and `combinaciones_especiales`.
2. Identify reward patterns (`numero_premiado`, `letras`, `monto`) and `vendedor`.

In [9]:
premios_data = []

for df in dataframes:
    body = df["body"]
    for line in body:
        match = re.match(r"(\d+)\s+(\w+)\s+\.+\s+([\d,]+\.?\d*)", line)
        if match:
            numero_premiado, letras, monto = match.groups()
            vendedor = None
            if "VENDIDO POR" in line:
                vendedor = line.split("VENDIDO POR")[1].strip()
                
            premios_data.append({
                "id_sorteo": df["filename"], # # You can map this with `numero_sorteo`
                "numero_premiado": numero_premiado,
                "letras": letras,
                "monto": monto.replace(",", ""),
                "vendido_por": vendedor
            })
premios_df = pd.DataFrame(premios_data)
premios_df['id_sorteo'] = premios_df['id_sorteo'].str.extract(r'no\._(\d+)', expand=False)
premios_df

,id_sorteo,numero_premiado,letras,monto,vendido_por
0,390,00044,P,600.00,None
1,390,00080,DT,600.00,None
2,390,00129,TT,1000.00,None
3,390,00180,DT,600.00,None
4,390,00228,P,800.00,None
...,...,...,...,...,...
8822,3052,79308,P,1000.00,None
8823,3052,79421,P,700.00,None
8824,3052,79733,TTR,570.00,None
8825,3052,79823,TTR,670.00,None
